
На основании нормативов по качеству питьевой воды и прайс листа с ценами бутилированной воды создайте нейро-консультанта, который бы отвечал на вопросы по данной информации. Для этого скачайте pdf файлы по ссылкам:

Нормативы: https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/water_standard.pdf

https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/water_requirements.pdf

Прайс-лист: https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/price_list.pdf

Документы с нормативами объедините в 1 БЗ, а из прайс-листа сделайте вторую БЗ.

Нейро-консультант должен на основании вопроса пользователя динамически выбирать соответствующую базу знаний и отвечать на оcнове нее на вопрос.

Проверьте работу нейро-консультанта на самостоятельно-сформулированных вопросах.

In [1]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai==1.55.3 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12 langchain-text-splitters==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


In [2]:
# импортируем необходимые библиотеки
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os
import re
import requests
import openai
import tiktoken
import matplotlib.pyplot as plt
from textwrap import fill
from langchain_openai import ChatOpenAI                   # класс ChatOpenAI
from langchain.prompts import PromptTemplate              # шаблона запроса
from langchain.chains import LLMChain                     # цепочка LLM
from langchain.chains.router import MultiRetrievalQAChain # Импорт многократного поиска

import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)

In [3]:
from google.colab import userdata

# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [4]:
# функция принимает URL PDF-файла и путь, по которому файл будет сохранен в файловой системе Colab
# она скачивает файл и сохраняет его, если HTTP-запрос успешен
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")

In [5]:
!pip install PyMuPDF==1.25.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 62.0 MB/s eta 0:00:00


In [6]:
# импорт модулей
import fitz # работа с документами в формате PDF

# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names):
    pdf_folder_path = "/content/"

    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_folder_path, pdf_file)

        # Извлечение текста из PDF
        try:
            with fitz.open(pdf_path) as pdf:
                text = ""
                for page in pdf:
                    text += page.get_text()
        except Exception as e:
            print(f"Ошибка при обработке файла {pdf_file}: {e}")
            continue

        # Разделение текста на части
        documents = text_splitter.split_text(text)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(all_documents, embeddings).as_retriever()

    return all_documents, vectorstore

In [25]:
url = "https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/water_standard.pdf"
save_path = "/content/water_standard.pdf"
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/water_standard.pdf


In [26]:
url = "https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/water_requirements.pdf"
save_path = "/content/water_requirements.pdf"
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/water_requirements.pdf


In [49]:
pdf_names = ['water_standard.pdf', 'water_requirements.pdf']
text_db_normatove, db_normatove = create_faiss_vectorstore(pdf_names)

In [50]:
text_db_normatove

['2.6. Общие требования к методам \nконтроля\nГОСТ Р 5 1 2 3 2 -9 8\nГОСУДАРСТВЕННЫ Й СТАНДАРТ РОССИЙСКОЙ Ф ЕДЕРАЦ И И\nВОДА ПИТЬЕВАЯ\nОбщие требования к организации \nи методам контроля качества\nИздание официальное\nГОССТАНДАРТ РОССИИ \nМ о с к в а\nобязательная сертификация услуг\nГОСТ Р 51232-98\nПредисловие\n1 РАЗРАБОТАН Техническим комитетом по стандартизации ТК 343 «Ка\xad\nчество воды» (ВНИИстандарт, МосводоканалНИИпроект, ГУП ЦП КВ, \nУНИИМ, НИИЭЧГО им. А.Н. Сысина, ГИЦПВ)\nВНЕСЕН Управлением Агролегпрома и химической продукции Госстандарта \nРоссии\n2 ПРИНЯТ И ВВЕДЕН В ДЕЙСТВИЕ Постановлением Госстандарта Рос\xad\nсии от 17 декабря 1998 г. № 449\n3 ВВЕДЕН ВПЕРВЫЕ\n4 ПЕРЕИЗДАНИЕ. Июнь 2005 г.\n© ИПК Издательство стандартов, 1999 \n© СТАНДАРТИНФОРМ, 2005 \n© СТАНДАРТИНФОРМ, 2008',
 '© ИПК Издательство стандартов, 1999 \n© СТАНДАРТИНФОРМ, 2005 \n© СТАНДАРТИНФОРМ, 2008\nНастоящий стандарт не может быть полностью или частично воспроизведен, \nтиражирован и распространен в качестве

In [51]:
url = 'https://storage.yandexcloud.net/aiueducation/Content/GPT_Professional/L9/price_list.pdf'
save_path = '/content/price_list.pdf'
download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/price_list.pdf


In [52]:
pdf_names = ['price_list.pdf']
text_db_price_list, db_price_list = create_faiss_vectorstore(pdf_names)

In [53]:
# посмотрим содержимое базы знаний
text_db_price_list

['на\n01  января  \n2019год\n штука\n упаковка\n1\nВода "Аква-Вита" газ  \n0,5л\n12\n11,00\n132,00\n2\nВода "Аква-Вита" негаз\n0,5л\n12\n11,00\n132,00\n3\nВода "Аква-Вита"(крышка Спорт) \nнегаз \n0,5л\n12\n12,00\n144,00\n4\nВода "Моя вода"  газ \n0,5л\n12\n11,50\n138,00\n5\nВода "Моя вода" негаз\n0,5л\n12\n11,50\n138,00\n6\nВода "Прима Аква" газ \n0,5л\n12\n11,80\n141,60\n7\nВода "Прима Аква" негаз  \n0,5л\n12\n11,80\n141,60\n8\nВода "Аква-Вита" газ \n1,5л\n6\n12,50\n75,00\n9\nВода "Аква-Вита" негаз \n1,5л\n6\n12,50\n75,00\n10\nВода "Моя вода" газ\n1,5л\n6\n12,00\n72,00\n11\nВода "Моя вода" негаз \n1,5л\n6\n12,00\n72,00\n12\nВода дистилированная\n1,5л\n6\n22,00\n132,00\n13\nВода "Аква-Вита" негаз \n5л\n2\n32,00\n64,00\n14\nВода "Дистилированная" \n5л\n2\n48,00\n96,00\n15\nВода "Моя вода" негаз \n5л\n2\n28,00\n56,00\nВода минеральная природная \nпитьевая столовая  « Аква Вита»',
 '5л\n2\n28,00\n56,00\nВода минеральная природная \nпитьевая столовая  « Аква Вита» \n1 состав (синие крышки)

In [55]:
retriever_infos = [
    {
        "name": "Normatives",
        "description": "Suitable for answering questions about General requirements for control methods AND Hygienic requirements for the water quality of centralized drinking water supply systems. Quality control",  # описываем, для каких вопросов применять данную БЗ
        "retriever": db_normatove
    },
    {
        "name": "Price List",
        "description": "Suitable for answering questions about price list", # описываем, для каких вопросов применять данную БЗ
        "retriever": db_price_list
    }
]

In [56]:
system = """
  Очень подробно и детально ответь на вопрос пользователя,
  опираясь точно на документ с информацией для ответа клиенту.
  Не придумывай ничего от себя. Не ссылайся на сами отрывки документа
  с информацией для ответа, клиент о них ничего не должен знать.
"""

In [57]:
# определяем модель
llm = ChatOpenAI(temperature=0)

# шаблон для запроса
prompt = PromptTemplate(input_variables=["query"], template="Question: {query}")

# цепочка, которую будем использовать по умолчанию
default_chain_llm = LLMChain(llm=llm, prompt=prompt)

# создание MultiRetrievalQAChain с указанием default_chain_llm
chain = MultiRetrievalQAChain.from_retrievers(
  llm=llm,
  retriever_infos=retriever_infos,
  default_chain_llm=default_chain_llm,
  verbose=True
)

In [58]:
print(chain.invoke("При каких условиях допускаются отклонения от гигиенических отклонений?")["result"])



> Entering new MultiRetrievalQAChain chain...
Normatives: {'query': 'При каких условиях допускаются отклонения от гигиенических отклонений?'}
> Finished chain.
Отклонения от гигиенических нормативов качества питьевой воды допускаются при выполнении следующих условий:
1. Обеспечение населения питьевой водой не может быть достигнуто иным способом.
2. Соблюдение согласованных с центром госсанэпиднадзора на ограниченный период времени максимально допустимых отклонений от гигиенических нормативов.
3. Максимальное ограничение срока действия отступлений.
4. Отсутствие угрозы здоровью населения в период действия отклонений.
5. Обеспечение информации населения о введении отклонений и сроках их действия, об отсутствии риска для здоровья, а также о рекомендациях по использованию питьевой воды.


In [60]:
print(chain.invoke("Сколько стоит негаз 'моя вода'")["result"])



> Entering new MultiRetrievalQAChain chain...
Price List: {'query': "Сколько стоит негаз 'моя вода'"}
> Finished chain.
Негазированная "Моя вода" стоит 11,50 рублей за бутылку объемом 0,5л и 12,00 рублей за бутылку объемом 1,5л.


In [61]:
print(chain.invoke("Где проводят отбор проб воды из распределительной сети?")["result"])



> Entering new MultiRetrievalQAChain chain...
Normatives: {'query': 'Где проводят отбор проб воды из распределительной сети?'}
> Finished chain.
Отбор проб воды из распределительной сети проводят из уличных водоразборных устройств на основных магистральных линиях, на наиболее возвышенных и тупиковых участках, а также из кранов внутренних водопроводных сетей домов.
